# Part 1 : Implementing QPE

## To hand out

1. A **report** with your answers in a **PDF FILE** (made out of LaTeX, libreoffice, ...)
  * Math and text answers
  * The code for the circuits
  * Screenshot of figures/circuits
  * python answers and results of runs
  * *etc*
  
2. **This notebook**
  * as a runnable script

But first, some libraries to load (nothing to modify here)

In [1]:
import numpy as np
from math import pi, gcd
try:
    from qiskit import *
    from matplotlib.pyplot import plot,show
except:
    ! python -m pip install matplotlib pylatexenc
    ! python -m pip install qiskit
from qiskit import *
from qiskit.circuit import *
from qiskit.extensions import *
from qiskit.circuit.library import *
from qiskit.extensions.simulator.snapshot import snapshot
from qiskit.quantum_info.operators import Operator
from qiskit.extensions.simulator.snapshot import snapshot
from qiskit.compiler import transpile
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller
from scipy import optimize
from matplotlib.pyplot import plot,show
%matplotlib inline
%config InlineBackend.figure_format = 'svg' # Makes the images look nice

# 0 - Before anything else

### Make sure you checked the notebook TP-Intro-QisKit


# 1 - Small practice

To start, check that when we measure $\frac1{\sqrt2}(|000\rangle+|111\rangle)$ we indeed get 000 and 111 about the same number of times. 

In [14]:
# Generating the circuit

q = QuantumRegister(3)   # We need 3 qubits..
c = ClassicalRegister(3) # ... and 3 bits to store the results
qc = QuantumCircuit(q,c) # the circuit !

qc.h(q[0])          # build a 

qc.cnot(q[0],q[1])  # Bell state
qc.cnot(q[0],q[2])  # Bell state


qc.measure(q, c) # measure of all of q


backend = BasicAer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1024)
res = dict(job.result().get_counts(qc))

qc.draw(output='latex_source', filename="rapport/ex1.tex")

res

{'000': 523, '111': 501}

#### Write record what you did and what you see in the separate report

# 2 - QPE

We've seen the QPE algorithm in the course, and you checked it worked with 3 qubits. Here we are going to implement it with the following unitary:

In [16]:
U = UnitaryGate(
    Operator([[1,0,0,0],
              [0,1,0,0],
              [0,0,1,0],
              [0,0,0,np.exp(pi*2j*(10/16))]]), label="U")

## Questions 

###  Q 2.1 Math questions

* What is doing this operator ? (`2j` is in Python the complex number $2\cdot i$)
    * $U |00\rangle = |00\rangle$
    * $U |01\rangle = |01\rangle$
    * $U |10\rangle = |10\rangle$
    * $U |11\rangle = e^{2j \pi \frac 6 8}|11\rangle$
* On how many qubits does it act ?
    * 2
* What are its eigenvalues/eigenvectors ?
    * $|00\rangle$ | 1
    * $|01\rangle$ | 1
    * $|10\rangle$ | 1
    * $|11\rangle$ | $e^{2j \pi \frac 6 8}$
* For each eigenvector, what should QPE return with 3 bits of precisions, as seen in the course ?
    * $|00\rangle$ -> 0
    * $|01\rangle$ -> 0
    * $|10\rangle$ -> 0
    * $|11\rangle$ -> 6

#### Give answers and explanations in the separate report 

### Q 2.2 Implementing QPE

Below a template to fill in for
- realize QPE with 3 bits of precision.
- on the eigenvector of non-trivial eigenvalue

We initialized a quantum circuit with 3 registers:
 - `eig` for storing the eigenvalues
 - `phi` for storing the eigenvector
 - `ceig` for storing the result of the measurement of the eigenvalue-register.
 
Note that we only need to measure the eigenvalues!

What you will need:
 - `QFT(size)` build for you a QFT on `size` qubits.
 - `U.control()` for controlling a gate `U`. The control qubit should be placed first in the list of wires.
 - `U.inverse()` for the inverse of the gate `U`.
 - `U.power(p)` add `p` times `U` on the circuit.
 - `qc.append(U, list_of_qubits)` applies the gate `U` on the list of qubits.
 - Beware : `phi` (for instance) is not a list but a register. So if you want to concatenate it with something else, you first have to make a list out of it with `list(phi)`.

In [19]:
size_eig = 5
size_phi = 2

eig = QuantumRegister(size_eig, name="eig")
phi = QuantumRegister(size_phi, name="phi")
ceig = ClassicalRegister(size_eig, name="ceig")
#cphi = ClassicalRegister(size_phi, name="cphi")
qc = QuantumCircuit(eig,phi,ceig)#,cphi)

qc.x(phi[0])
qc.x(phi[1])
#qc.cnot(phi[0], phi[1])
for i in range(0, size_eig):
    qc.h(eig[i])
    qc.append(U.power(2**i).control(), [eig[i], phi[0], phi[1]])

qc.append(QFT(size_eig).inverse(), eig)
qc.measure(eig, ceig)
#qc.measure(phi, cphi)



qc.draw()

# First, make sure that the drawing is OK.

┌───┐                                                        »
 eig_0: ┤ H ├──────■─────────────────────────────────────────────────»
        ├───┤      │                                                 »
 eig_1: ┤ H ├──────┼─────────────■───────────────────────────────────»
        ├───┤      │             │                                   »
 eig_2: ┤ H ├──────┼─────────────┼─────────────■─────────────────────»
        ├───┤      │             │             │                     »
 eig_3: ┤ H ├──────┼─────────────┼─────────────┼─────────────■───────»
        ├───┤      │             │             │             │       »
 eig_4: ┤ H ├──────┼─────────────┼─────────────┼─────────────┼───────»
        ├───┤┌─────┴──────┐┌─────┴──────┐┌─────┴──────┐┌─────┴──────┐»
 phi_0: ┤ X ├┤0           ├┤0           ├┤0           ├┤0           ├»
        ├───┤│  unitary^1 ││  unitary^2 ││  unitary^4 ││  unitary^8 │»
 phi_1: ┤ X ├┤1           ├┤1           ├┤1           ├┤1           ├»
        └───┘└────────────┘└────────────┘└────────────┘└────────────┘»
ceig: 5/═════════════════════════════════════════════════════════════»
                                                                     »
«                       ┌───────┐┌─┐            
« eig_0: ───────────────┤0      ├┤M├────────────
«                       │       │└╥┘┌─┐         
« eig_1: ───────────────┤1      ├─╫─┤M├─────────
«                       │       │ ║ └╥┘┌─┐      
« eig_2: ───────────────┤2 IQFT ├─╫──╫─┤M├──────
«                       │       │ ║  ║ └╥┘┌─┐   
« eig_3: ───────────────┤3      ├─╫──╫──╫─┤M├───
«                       │       │ ║  ║  ║ └╥┘┌─┐
« eig_4: ───────■───────┤4      ├─╫──╫──╫──╫─┤M├
«        ┌──────┴──────┐└───────┘ ║  ║  ║  ║ └╥┘
« phi_0: ┤0            ├──────────╫──╫──╫──╫──╫─
«        │  unitary^16 │          ║  ║  ║  ║  ║ 
« phi_1: ┤1            ├──────────╫──╫──╫──╫──╫─
«        └─────────────┘          ║  ║  ║  ║  ║ 
«ceig: 5/═════════════════════════╩══╩══╩══╩══╩═
«                                 0  1  2  3  4

In [20]:
# Then run the backend !
qc.draw(output='latex_source', filename="rapport/ex2.tex")

backend = BasicAer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1024)
job.result().get_counts(qc)

{'10100': 1024}

### Q 2.3 Exact result

- (a) Is it the expected result ? oui
- (b) Change the $\frac68$ of the phase of $U$: use $\frac18$, then $\frac28$... Is QPE returning the correct answer?
- (c) Change the precision : use $4$ qubits for `eig`, and change the fraction in the phase of $U$ to $\frac{10}{16}$ : is QPE indeed returning $10$ in binary ?
- (d) Move to $5$ bits of précision: is it still working ?

#### Give answers and explanations in the separate report 

### Q 2.4 Approximate result

Use  $\frac13$ in the phase of $U$:
- With 3 bits of precision
- With 4 bits of precision
- With 5 bits of precision

**Question** What do you observe? Can you explain it? What do you read?

#### Give answers and explanations in the separate report 

### Q 2.5 Superposition

We saw that the circuit of QPE has no problem with a superposition of eigenvectors. Try to change the initialization of `phi` with 
$$
\frac1{\sqrt2}(|\phi_1\rangle + |\phi_2\rangle),
$$
two eigenvectors of $U$ (one with trivial eigenvalue, the other one non-trivial).

Also measure the register `phi` at the end of the circuit, and analyze the result: can you explain what you see?

Try this experiment with phase $\frac38$ and $\frac13$.

#### Give answers and explanations in the separate report 